In [ ]:
#transformation de fourier discrete
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import cmath
pi2 = cmath.pi * 2.0
def DFT(fnList):
    N = len(fnList)
    FmList = []
    for m in range(N):
        Fm = 0.0
        for n in range(N):
            Fm += fnList[n] * cmath.exp(- 1j * pi2 * m * n / N)
        FmList.append(Fm / N)
    return FmList

In [ ]:
import numpy as np



def wavefront_initialize(pixelsize_h=1e-6,pixelsize_v=1e-6,npixels_h=1024,npixels_v=1024,amplitude_value=0.0):

   
    amplitude = np.zeros((npixels_h,npixels_v))  

    amplitude += amplitude_value

    p_i_h = np.arange(npixels_h) * pixelsize_h

    p_x = (p_i_h - 0.5 * (p_i_h[-1] - p_i_h[0]) )

    p_i_v = np.arange(npixels_v) * pixelsize_v

    p_y = (p_i_v - 0.5 * (p_i_v[-1] - p_i_v[0]) )

    return p_x,p_y,amplitude



def wavefront_aperture(p_x,p_y,amplitude,diameter=40e-6):

   
    p_xx = p_x[:, np.newaxis]

    p_yy = p_y[np.newaxis, :]



    filter = np.zeros_like(amplitude)



    radius = (diameter/2)

    print("radius=%f um"%(1e6*radius))

    filter_illuminated_indices = np.where( (np.abs(p_xx) < radius) & (np.abs(p_yy) < radius))

    if filter_illuminated_indices[0].size ==0:

            print("Warning: wavefront_aperture(): Nothing goes trough the aperture")

    else:

            filter[filter_illuminated_indices] = 1.0

    



    return p_x,p_y,amplitude*filter



def propagator2d_fraunhoffer(p_x,p_y,image,wavelength=1e-10):




   

    F1 = np.fft.fft2(image)  
    
    F2 = np.fft.fftshift( F1 )



    

    pixelsize = p_x[1] - p_x[0]

    npixels = p_x.size

    freq_nyquist = 0.5/pixelsize

    freq_n = np.linspace(-1.0,1.0,npixels)

    freq_x = freq_n * freq_nyquist

    freq_x *= wavelength

    
    pixelsize = p_y[1] - p_y[0]

    npixels = p_y.size

    freq_nyquist = 0.5/pixelsize

    freq_n = np.linspace(-1.0,1.0,npixels)

    freq_y = freq_n * freq_nyquist

    freq_y *= wavelength



    return freq_x,freq_y,F2





 

def line_image(image,horizontal_or_vertical='H'):

    if horizontal_or_vertical == "H":

        npixels = image.shape[0]

        tmp = image[:,image.shape[1]/2]

    else:

        npixels = image.shape[1]

        tmp = image[image.shape[0]/2,:]

    return tmp



def plot_show():



    import matplotlib.pylab as plt



    plt.show()



def plot_image(mymode,theta,psi,title="TITLE",xtitle=r"X [$\mu m$]",ytitle=r"Y [$\mu m$]",cmap=None,show=1):



    import matplotlib.pylab as plt



    fig = plt.figure()

   
    plt.imshow(mymode.T,origin='lower',extent=[theta[0],theta[-1],psi[0],psi[-1]],cmap=cmap)

    plt.colorbar()

    ax = fig.gca()

    ax.set_xlabel(xtitle)

    ax.set_ylabel(ytitle)



    plt.title(title)



    if show: plt.show()



def plot(*positional_parameters,title="",xtitle="",ytitle="",show=1,legend=None,color=None):



    import matplotlib.pylab as plt



    n_arguments = len(positional_parameters)

    if n_arguments == 0:

        return



    fig = plt.figure()



    if n_arguments == 1:

        y = positional_parameters[0]

        x = np.arange(y.size)

        plt.plot(x,y,label=legend)

    elif n_arguments == 2:

        x = positional_parameters[0]

        y = positional_parameters[1]

        plt.plot(x,y,label=legend,color=color)

    elif n_arguments == 4:

        x1 = positional_parameters[0]

        y1 = positional_parameters[1]

        x2 = positional_parameters[2]

        y2 = positional_parameters[3]

        if legend != None:

            legend1 = legend[0]

            legend2 = legend[1]

        else:

            legend1 = None

            legend2 = None

        if color != None:

            color1 = color[0]

            color2 = color[1]

        else:

            color1 = None

            color2 = None

        plt.plot(x1,y1,label=legend1,color=color1)

        plt.plot(x2,y2,label=legend2,color=color2)

    else:

        "Incorrect number of arguments, plotting only two first arguments"

        x = positional_parameters[0]

        y = positional_parameters[1]

        plt.plot(x,y,label=legend)



    if legend != None:

        ax = plt.subplot(111)

        ax.legend(bbox_to_anchor=(1.1, 1.05))



    plt.title(title)

    plt.xlabel(xtitle)

    plt.ylabel(ytitle)





    if show:

        plt.show()



def main():

    

    wavelength        = 1.24e-10

    aperture_diameter = 40e-6 

   
    pixelsize_x = 1e-6

    pixelsize_y = pixelsize_x

    npixels_x =  1024    # 200 #

    npixels_y =  npixels_x # 50  #



    propagation_distance = 30.0 



    method = "fraunhofer"

   

    p_x,p_y,amplitude = wavefront_initialize(pixelsize_x,pixelsize_y,npixels_x,npixels_y,amplitude_value=1.0)

  

    p_x,p_y,amplitude = wavefront_aperture(p_x,p_y,amplitude,diameter=aperture_diameter)



    plot_image(np.abs(amplitude)**2,p_x*1e6,p_y*1e6, show=0,

               title="aperture intensity, Diameter=%5.1f um"%(1e6*aperture_diameter),xtitle="X [um]",ytitle="Y [um]")






    angle_x, angle_y, amplitude_propagated = propagator2d_fraunhoffer(p_x,p_y,amplitude,wavelength=wavelength)

    print(amplitude_propagated)


    plot_image(np.abs(amplitude_propagated)**2,angle_x*1e6,angle_y*1e6, show=0,

               title="Diffracted intensity (%s)"%method,xtitle="X [urad]",ytitle="Y [urad]")



    horizontal_intensity_profile = line_image(np.abs(amplitude_propagated)**2,horizontal_or_vertical='H')

    horizontal_intensity_profile /= horizontal_intensity_profile.max()



    vertical_intensity_profile = line_image(np.abs(amplitude_propagated)**2,horizontal_or_vertical='V')

    vertical_intensity_profile /= vertical_intensity_profile.max()
    print(vertical_intensity_profile)
     
    x = (2*np.pi/wavelength) * (aperture_diameter/2) * angle_x

    y = (2*np.pi/wavelength) * (aperture_diameter/2) * angle_y

    U_vs_theta_x = 2*np.sin(x)/x

    U_vs_theta_y = 2*np.sin(y)/y

    I_vs_theta_x = U_vs_theta_x**2

    I_vs_theta_y = U_vs_theta_y**2

    I_vs_theta_x /= I_vs_theta_x.max()

    I_vs_theta_y /= I_vs_theta_y.max()
  

    plot( angle_x*1e6, horizontal_intensity_profile, angle_x*1e6, I_vs_theta_x, show=0,

          legend=["profile","theory"],color=["red","black"],

          title="Horizontal profile of diffracted intensity (%s)"%method,xtitle='theta [urad]',ytitle='Diffracted intensity [a.u.]')

    plot( angle_y*1e6, vertical_intensity_profile, angle_y*1e6, I_vs_theta_y, show=1,

          legend=["profile","theory"],color=["red","black"],

          title="Vertical profile of diffracted intensity (%s)"%method,xtitle='theta [urad]',ytitle='Diffracted intensity [a.u.]')



if __name__ == "__main__":

    main()

radius=20.000000 um
[[ 0.00000000 +0.00000000e+00j  0.00000000 +0.00000000e+00j
   0.00000000 +0.00000000e+00j ...,  0.00000000 +0.00000000e+00j
   0.00000000 +0.00000000e+00j  0.00000000 +0.00000000e+00j]
 [ 0.00000000 +0.00000000e+00j -0.01498423 -9.19432521e-05j
   0.02974281 +2.73757106e-04j ...,  0.04405624 -2.70329086e-04j
  -0.02974393 +9.12535139e-05j  0.01498451 -8.88178420e-16j]
 [ 0.00000000 +0.00000000e+00j  0.02974281 +2.73757106e-04j
  -0.05903714 -7.24531141e-04j ..., -0.08745230 +2.68301136e-04j
   0.05904159 +0.00000000e+00j -0.02974393 -9.12535139e-05j]
 ..., 
 [ 0.00000000 +0.00000000e+00j  0.04405624 -2.70329086e-04j
  -0.08745230 +2.68301136e-04j ..., -0.12951346 +2.38432331e-03j
   0.08744242 -1.34145517e-03j -0.04405375 +5.40647994e-04j]
 [ 0.00000000 +0.00000000e+00j -0.02974393 +9.12535139e-05j
   0.05904159 -8.88178420e-16j ...,  0.08744242 -1.34145517e-03j
  -0.05903714 +7.24531141e-04j  0.02974281 -2.73757106e-04j]
 [ 0.00000000 +0.00000000e+00j  0.01498451 

C:\Users\atefe\Anaconda3\lib\site-packages\ipykernel\__main__.py:115: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\atefe\Anaconda3\lib\site-packages\ipykernel\__main__.py:121: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future



[  0.00000000e+00   9.36532153e-06   3.69009942e-05 ...,   8.09596313e-05
   3.69009942e-05   9.36532153e-06]